In [2]:
# + tags=[]
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import box, Point
%matplotlib inline

In [3]:
# + tags=[]
csv_path = "data/raw/ais/aisdk-2024-03-01.csv"
# Wir lesen nur die ersten 100 000 Zeilen zum schnellen Test
df = pd.read_csv(csv_path, parse_dates=['# Timestamp'], nrows=100_000)
print("Spalten:", df.columns.tolist())
df.head()

Spalten: ['# Timestamp', 'Type of mobile', 'MMSI', 'Latitude', 'Longitude', 'Navigational status', 'ROT', 'SOG', 'COG', 'Heading', 'IMO', 'Callsign', 'Name', 'Ship type', 'Cargo type', 'Width', 'Length', 'Type of position fixing device', 'Draught', 'Destination', 'ETA', 'Data source type', 'A', 'B', 'C', 'D']


,# Timestamp,Type of mobile,MMSI,Latitude,Longitude,Navigational status,ROT,SOG,COG,Heading,...,Length,Type of position fixing device,Draught,Destination,ETA,Data source type,A,B,C,D
0,2024-01-03,Class A,219000873,56.990910,10.304543,Under way using engine,NaN,0.0,30.2,NaN,...,NaN,Undefined,NaN,Unknown,NaN,AIS,NaN,NaN,NaN,NaN
1,2024-01-03,Base Station,2190068,56.447260,10.945872,Unknown value,NaN,NaN,NaN,NaN,...,NaN,GPS,NaN,Unknown,NaN,AIS,NaN,NaN,NaN,NaN
2,2024-01-03,Class A,219016683,56.800165,9.024933,Under way using engine,0.0,0.0,257.3,17.0,...,NaN,Undefined,NaN,Unknown,NaN,AIS,NaN,NaN,NaN,NaN
3,2024-01-03,Class A,219000615,56.967093,9.224287,Restricted maneuverability,0.0,2.3,69.8,67.0,...,NaN,Undefined,NaN,Unknown,NaN,AIS,NaN,NaN,NaN,NaN
4,2024-01-03,Base Station,2190071,57.110043,8.648282,Unknown value,NaN,NaN,NaN,NaN,...,NaN,GPS,NaN,Unknown,NaN,AIS,NaN,NaN,NaN,NaN


In [4]:
# + tags=[]
# BBox aus GeoJSON (vom Plotseamap-Buffer) laden
bbox = gpd.read_file(...).to_crs("EPSG:4326")
clip_poly = bbox.geometry.unary_union  # oder bbox.geometry.union_all()

# nach dem Einlesen, direkt umbenennen:
df = df.rename(columns={'Latitude':'lat', 'Longitude':'lon'})

# und dann:
gdf = gpd.GeoDataFrame(
    df,
    geometry=gpd.points_from_xy(df['lon'], df['lat']),
    crs="EPSG:4326"
)

# Clip auf unser Untersuchungsgebiet
gdf = gdf[gdf.geometry.within(bbox)]
gdf = gdf.to_crs(epsg=3857)
print(f"Gefiltert auf BBox: {len(gdf)} Punkte")
gdf.sample(5)

/var/folders/xp/7528x0fs5fzcfyztpy9jt7x80000gn/T/ipykernel_42614/1590490835.py:4: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  .to_crs("EPSG:4326").unary_union


AttributeError: 'DataFrame' object has no attribute 'lon'